# ETL-Pipeline zum Import von Sensor-Daten

## Aufgabenstellung
Firma: Peter Döhle Schiffahrts-KG

Daten: Signaldaten von 12+ Schiffen
- Treibstoff Emissionen, Tankfüllstand, Geschwindigkeit, Gewicht,...
- im 15 min Takt via Rest API verfügbar

ursprüngliches Ziel: Skalierbare ETL-Strecke für den poc von Microsoft Fabric

geändertes Ziel: Vergleich verschiedener ETL-Ansätze anhand festgelegter Kriterien


## API-Endpoints

Response: JSON-Format
- /fleet : Alle Schiffe inkl. technischer Informationen
- /fleet/{imo}/signals: Auflistung der für das Schiff verfügbaren Sensordaten inkl. Erklärung
- /fleet/{imo}/timeseries?{optionaleParameter}: Zeitreihe der Sensordaten des Schiffes in 5-min Abständen

Zielbild: 
Zeitreihe mit verständlichen Signalnamen pivotisiert und vollständig historisiert

## Herausforderungen

- Teils starke Verschachtelung
- Schlechte Verbindung des Schiffes 
    - Fehlende Daten als NULL gespeichert
- Fehlende Daten teils nachträglich überschrieben 
    - überlappendes Laden notwendig
- Veränderung der Anzahl der Signale 
- Pivotierung führt zu sehr vielen Spalten

## Setup 

```bash
# Virtuelle Umgebung erstellen
python -m venv hoppe-env

# Umgebung aktivieren
source hoppe-env/Scripts/activate

# Pakete installiereb
pip install -r requirements.txt --quiet
```

In [1]:
# Importieren der ETL-Pipeline-Klassen
import os
import logging
import argparse
import json
from pathlib import Path
from datetime import datetime, timedelta, timezone
from dotenv import load_dotenv
import pandas as pd
import polars as pl
import sqlalchemy as sa
from sqlalchemy.sql import text
from dataclasses import dataclass
import requests
from typing import Dict, Optional, Tuple, List, Union
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from concurrent.futures import ThreadPoolExecutor


In [2]:
# Hauptordner erstellen
os.makedirs('./data', exist_ok=True)

# Unterordner erstellen
for sub_dir in ['raw_data', 'transformed_data', 'gaps_data']:
    os.makedirs(os.path.join('../data', sub_dir), exist_ok=True)

In [3]:
# Logging einrichten
def setup_logging():
    """Konfiguriert das Logging"""
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
        handlers=[
            logging.FileHandler("pipeline.log"),
            logging.StreamHandler()
        ]
    )
    return logging.getLogger("hoppe_etl_pipeline")

In [4]:
@dataclass
class Config:
    """Configuration settings for the data pipeline"""
    base_url: str = "https://api.hoppe-sts.com/"
    raw_path: str = "./data/raw_data"
    transformed_path: str = "./data/transformed_data"
    gaps_path: str = "./data/gaps_data"  # Neuer Pfad für Null-Wert-Lücken
    batch_size: int = 1000
    max_workers: int = 8  # Erhöhte Worker für bessere Parallelisierung
    retry_attempts: int = 5  # Erhöhte Retry-Versuche
    timeout: int = 45  # Erhöhter Timeout
    days_to_keep: int = 90  # Daten werden für 90 Tage aufbewahrt
    history_days: int = 5  # Letzten 5 Tage für Historie laden

In [5]:
class APIClient:
    """Handles API communication with retry logic"""
    
    def __init__(self, base_url: str, api_key: str, timeout: int = 30):
        self.base_url = base_url
        self.api_key = api_key
        self.session = self._create_session(timeout)

    def _create_session(self, timeout: int) -> requests.Session:
        """Creates requests session with retry logic"""
        session = requests.Session()
        retry_strategy = Retry(
            total=5,
            backoff_factor=1,
            status_forcelist=[429, 500, 502, 503, 504],
            allowed_methods=["GET"]
        )
        adapter = HTTPAdapter(max_retries=retry_strategy)
        session.mount("http://", adapter)
        session.mount("https://", adapter)
        session.headers.update({
            "Authorization": f"ApiKey {self.api_key}",
            "Accept": "application/json"
        })
        session.timeout = timeout
        return session
    
    def get_data(self, relative_url: str, params: Optional[Dict] = None) -> Tuple[requests.Response, Optional[dict]]:
        """
        Fetches data from API with error handling
        
        Args:
            relative_url: API endpoint path
            params: Optional query parameters
            
        Returns:
            Tuple of (Response, JSON data)
        """
        try:
            request_url = f"{self.base_url}{relative_url}"
            response = self.session.get(request_url, params=params)
            response.raise_for_status()
            return response, response.json()
        except requests.exceptions.RequestException as e:
            logger.error(f"API request failed: {str(e)}")
            if hasattr(e, 'response'):
                return e.response, None
            return None, None

In [6]:
class DataStorage:
    """Handles data storage operations"""
    
    def __init__(self, config):
        self.config = config

    def write_file(
        self, 
        data: Union[List, Dict, pl.DataFrame],
        filename: str,
        path: str,
        postfix: str
    ) -> None:
        """Writes data to file system"""
        os.makedirs(path, exist_ok=True)
        full_path = f"{path}/{filename}.{postfix}"
        
        try:
            if postfix == 'json':
                with open(full_path, 'w') as f:
                    json.dump(data, f)
            elif postfix == 'parquet':
                if not isinstance(data, pl.DataFrame):
                    if isinstance(data, list) or isinstance(data, dict):
                        data = pl.DataFrame(data)
                    else:
                        raise ValueError("Data must be DataFrame, List, or Dict for parquet format")
                data.write_parquet(full_path, compression="snappy")
            else:
                raise ValueError(f"Unsupported format: {postfix}")
                
            logger.info(f"Data saved to {full_path}")
        except Exception as e:
            logger.error(f"Failed to write file {full_path}: {str(e)}")
            raise

    def cleanup_old_data(self, base_path: str, days_to_keep: int = 90) -> None:
        """
        Löscht Daten, die älter als days_to_keep Tage sind
        """
        try:
            today = datetime.now()
            cutoff_date = today - timedelta(days=days_to_keep)
            
            # Wandle in Pfad-Format um (Jahr/Monat/Tag)
            cutoff_path = cutoff_date.strftime('%Y/%m/%d')
            base_path = Path(base_path)
            
            if not base_path.exists():
                return
                
            # Durchsuche alle Jahresordner
            for year_dir in base_path.glob("*"):
                if not year_dir.is_dir() or not year_dir.name.isdigit():
                    continue
                    
                year = int(year_dir.name)
                
                # Überspringe Ordner, die definitiv behalten werden sollen
                if year > cutoff_date.year:
                    continue
                
                # Behandle Jahre, die teilweise gelöscht werden müssen
                if year == cutoff_date.year:
                    for month_dir in year_dir.glob("*"):
                        if not month_dir.is_dir() or not month_dir.name.isdigit():
                            continue
                            
                        month = int(month_dir.name)
                        
                        # Überspringe Monate, die definitiv behalten werden sollen
                        if month > cutoff_date.month:
                            continue
                            
                        # Behandle Monate, die teilweise gelöscht werden müssen
                        if month == cutoff_date.month:
                            for day_dir in month_dir.glob("*"):
                                if not day_dir.is_dir() or not day_dir.name.isdigit():
                                    continue
                                    
                                day = int(day_dir.name)
                                
                                # Lösche Tage, die älter als der Cutoff sind
                                if day < cutoff_date.day:
                                    logger.info(f"Removing old data directory: {day_dir}")
                                    # In Produktion wäre hier tatsächliches Löschen (shutil.rmtree)
                                    # Für Sicherheit vorerst nur Logging
                                    # import shutil
                                    # shutil.rmtree(day_dir)
                        
                        # Lösche den gesamten Monat, wenn er älter als der Cutoff-Monat ist
                        elif month < cutoff_date.month:
                            logger.info(f"Removing old data directory: {month_dir}")
                            # import shutil
                            # shutil.rmtree(month_dir)
                
                # Lösche das gesamte Jahr, wenn es älter als das Cutoff-Jahr ist
                elif year < cutoff_date.year:
                    logger.info(f"Removing old data directory: {year_dir}")
                    # import shutil
                    # shutil.rmtree(year_dir)
                    
            logger.info(f"Cleanup of data older than {cutoff_date.strftime('%Y-%m-%d')} completed")
                
        except Exception as e:
            logger.error(f"Error during data cleanup: {str(e)}")

    def write_to_db(
        self,
        df: pl.DataFrame,
        engine: sa.Engine,
        table_name: str,
        if_exists: str = "replace",
        batch_size: int = 10000
    ) -> None:
        """Writes DataFrame to database in batches"""
        try:
            # Convert to pandas for writing to database in batches
            pdf = df.to_pandas()
            total_rows = len(pdf)
            
            if total_rows == 0:
                logger.warning(f"No data to write to table {table_name}")
                return
                
            logger.info(f"Writing {total_rows} rows to table {table_name}")
            
            # Write in batches to avoid memory issues
            for i in range(0, total_rows, batch_size):
                end = min(i + batch_size, total_rows)
                batch = pdf.iloc[i:end]
                
                # For first batch, replace or append based on if_exists parameter
                if i == 0:
                    batch.to_sql(
                        table_name,
                        engine,
                        if_exists=if_exists,
                        index=False,
                        method='multi',
                        chunksize=1000
                    )
                else:
                    # For subsequent batches, always append
                    batch.to_sql(
                        table_name,
                        engine,
                        if_exists='append',
                        index=False,
                        method='multi',
                        chunksize=1000
                    )
                    
                logger.info(f"Wrote batch {i//batch_size + 1} of {(total_rows-1)//batch_size + 1} to table {table_name}")
                
            logger.info(f"Data successfully written to table {table_name}")
        except Exception as e:
            logger.error(f"Database write failed: {str(e)}")
            raise
            
    def write_ts_to_msdb(
        self,
        df: pl.DataFrame, 
        engine: sa.Engine,
        batch_size: int = 10000
    ) -> None:
        """
        Writes time series data to MSSQL database using a staging table approach.
        
        This specialized method:
        1. Writes the dataframe to a staging table
        2. Ensures the main table exists with the correct schema
        3. Adds any missing columns to the main table
        4. Merges data from staging to the main table using MERGE statement
        
        Parameters:
            df (pl.DataFrame): Time series data to write
            engine (sa.Engine): SQLAlchemy engine for database connection
            batch_size (int): Number of rows to write in each batch
        """
        try:
            # If dataframe is empty, nothing to do
            if len(df) == 0:
                logger.warning("No time series data to write to database")
                return
            
            # Step 1: Write DataFrame to staging table
            self.write_to_db(df, engine, "TimeSeries_Staging", if_exists="replace", batch_size=batch_size)
            logger.info("Data written to TimeSeries_Staging table")
            
            # Step 2: Ensure main pivot table exists
            create_pivot_table_sql = """
            IF OBJECT_ID('TimeSeries_pivot', 'U') IS NULL
            BEGIN
                CREATE TABLE TimeSeries_pivot (
                    imo NVARCHAR(255) NOT NULL,
                    signal_timestamp DATETIME NOT NULL,
                    loaddate NVARCHAR(255),
                    PRIMARY KEY (imo, signal_timestamp)
                );
            END
            """
            
            with engine.connect() as conn:
                conn.execute(text(create_pivot_table_sql))
                conn.commit()
                logger.info("Ensured TimeSeries_pivot table exists")
            
            # Step 3: Add missing columns to main table
            add_columns_sql = """
            DECLARE @column_name NVARCHAR(255)
            DECLARE @sql NVARCHAR(MAX)

            DECLARE column_cursor CURSOR FOR
            SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS 
            WHERE TABLE_NAME = 'TimeSeries_Staging'
            AND COLUMN_NAME NOT IN (SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = 'TimeSeries_pivot')
            AND COLUMN_NAME NOT IN ('imo', 'signal_timestamp', 'loaddate')  -- Skip key columns and loaddate

            OPEN column_cursor
            FETCH NEXT FROM column_cursor INTO @column_name

            WHILE @@FETCH_STATUS = 0
            BEGIN
                SET @sql = 'ALTER TABLE TimeSeries_pivot ADD [' + @column_name + '] FLOAT NULL'
                EXEC sp_executesql @sql
                FETCH NEXT FROM column_cursor INTO @column_name
            END

            CLOSE column_cursor
            DEALLOCATE column_cursor
            """

            with engine.connect() as conn:
                conn.execute(text(add_columns_sql))
                conn.commit()
                logger.info("Added any missing columns to TimeSeries_pivot table")
            
            # Step 4: Pivotieren der Daten in der Staging-Tabelle
            # Dieses SQL transformiert die Daten von der Staging-Tabelle, wo sie im Format
            # imo, signal, signal_timestamp, signal_value sind, in das Pivot-Format
            pivot_staging_sql = """
            -- Create a temporary table to hold pivoted data
            IF OBJECT_ID('tempdb..#TempPivot', 'U') IS NOT NULL
                DROP TABLE #TempPivot;

            -- Get the list of unique signals for dynamic pivot
            DECLARE @columns NVARCHAR(MAX);
            DECLARE @sql NVARCHAR(MAX);

            -- Create a list of signals as columns for the PIVOT operation
            SELECT @columns = STRING_AGG(QUOTENAME(signal), ',')
            FROM (SELECT DISTINCT signal FROM TimeSeries_Staging) AS signals;

            -- Prepare the dynamic SQL for pivoting
            SET @sql = N'
            SELECT imo, signal_timestamp, loaddate, ' + @columns + '
            INTO #TempPivot
            FROM (
                SELECT imo, signal, signal_timestamp, signal_value, loaddate
                FROM TimeSeries_Staging
            ) AS src
            PIVOT (
                MAX(signal_value)
                FOR signal IN (' + @columns + ')
            ) AS pvt;
            ';

            -- Execute the dynamic SQL to create the temporary pivot table
            EXEC sp_executesql @sql;

            -- Create a MERGE statement to update the main table
            SET @sql = N'
            MERGE INTO TimeSeries_pivot AS target
            USING #TempPivot AS source
            ON target.imo = source.imo AND target.signal_timestamp = source.signal_timestamp
            WHEN MATCHED THEN
                UPDATE SET 
                    loaddate = source.loaddate' +
                    -- Add column updates from temp table, excluding key columns
                    (SELECT STRING_AGG(', ' + QUOTENAME(COLUMN_NAME) + ' = source.' + QUOTENAME(COLUMN_NAME), '')
                     FROM INFORMATION_SCHEMA.COLUMNS
                     WHERE TABLE_NAME = 'TimeSeries_pivot'
                     AND COLUMN_NAME NOT IN ('imo', 'signal_timestamp', 'loaddate')) + '
            WHEN NOT MATCHED THEN
                INSERT (imo, signal_timestamp, loaddate' +
                    -- Add columns from temp table, excluding key columns
                    (SELECT STRING_AGG(', ' + QUOTENAME(COLUMN_NAME), '')
                     FROM INFORMATION_SCHEMA.COLUMNS
                     WHERE TABLE_NAME = 'TimeSeries_pivot'
                     AND COLUMN_NAME NOT IN ('imo', 'signal_timestamp', 'loaddate')) + ')
                VALUES (source.imo, source.signal_timestamp, source.loaddate' +
                    -- Add values from temp table, excluding key columns
                    (SELECT STRING_AGG(', source.' + QUOTENAME(COLUMN_NAME), '')
                     FROM INFORMATION_SCHEMA.COLUMNS
                     WHERE TABLE_NAME = 'TimeSeries_pivot'
                     AND COLUMN_NAME NOT IN ('imo', 'signal_timestamp', 'loaddate')) + ');';

            -- Execute the merge
            EXEC sp_executesql @sql;

            -- Clean up
            DROP TABLE #TempPivot;
            """
            
            with engine.connect() as conn:
                conn.execute(text(pivot_staging_sql))
                conn.commit()
                logger.info("Data merged into TimeSeries_pivot table")
                
            # Optional: Schreibe Gaps-Daten in separate Tabelle
            gaps_table_sql = """
            IF OBJECT_ID('TimeSeries_Gaps', 'U') IS NULL
            BEGIN
                CREATE TABLE TimeSeries_Gaps (
                    imo NVARCHAR(255) NOT NULL,
                    signal NVARCHAR(255) NOT NULL,
                    gap_start DATETIME NOT NULL,
                    gap_end DATETIME NOT NULL,
                    loaddate NVARCHAR(255),
                    PRIMARY KEY (imo, signal, gap_start)
                );
            END
            """
            
            with engine.connect() as conn:
                conn.execute(text(gaps_table_sql))
                conn.commit()
                logger.info("Ensured TimeSeries_Gaps table exists")
                
        except Exception as e:
            logger.error(f"MSSQL database operation failed: {str(e)}")
            raise

In [7]:
class DataProcessor:
    """Processes raw API data into analytics-ready format"""

    @staticmethod
    def get_imo_numbers(data: List[dict]) -> List[str]:
        """Extracts IMO numbers from ship data"""
        return [ship['imo'] for ship in data if ship.get('active', True)]

    @staticmethod
    def transform_signals(signals: pl.DataFrame, run_timestamp: str) -> pl.DataFrame:
        """Transforms signals data"""
        if len(signals) == 0:
            return signals
            
        signals = (
            signals.lazy()
            .unnest("signals")
            .unpivot(index="imo", variable_name="signal")
            .unnest("value")
        )

        # Unnest remaining structs
        for column, dtype in signals.collect_schema().items():
            if dtype == pl.Struct:
                signals = signals.unnest(column)

        # Handle null columns
        for column, dtype in signals.collect_schema().items():
            if dtype == pl.Null:
                signals = signals.with_columns(pl.col(column).cast(pl.String))
        
        # Add loaddate
        signals = signals.with_columns(
            pl.lit(run_timestamp).alias("loaddate")
        )
                
        return signals.collect()
    
    @staticmethod
    def transform_timeseries(timeseries: pl.DataFrame, imo: str, run_timestamp: str) -> Tuple[pl.DataFrame, pl.DataFrame]:
        """
        Transforms time series data and extracts gaps data
        
        Returns:
            Tuple of (transformed_data, gaps_data)
        """
        if len(timeseries) == 0:
            return timeseries, pl.DataFrame()
        
        # Initial transformation
        transformed = (
            timeseries.lazy()
            .drop("timestamp")
            .unpivot(variable_name="signal")
            .unnest("value")
            .unpivot(
                index="signal",
                variable_name="signal_timestamp",
                value_name="signal_value",
            )
            .with_columns(
                pl.lit(imo).alias("imo"),
                pl.lit(run_timestamp).alias("loaddate")
            )
        )
        
        # Identifiziere Lücken (NULL-Werte)
        gaps = (
            transformed
            .filter(pl.col("signal_value").is_null())
            .select(["imo", "signal", "signal_timestamp", "loaddate"])
            .with_columns(
                pl.col("signal_timestamp").alias("gap_start")
            )
        )
        
        # Entferne NULL-Werte aus dem Hauptdatensatz
        data = transformed.filter(pl.col("signal_value").is_not_null())
        
        return data.collect(), gaps.collect()
    
    @staticmethod
    def process_gaps(gaps_df: pl.DataFrame) -> pl.DataFrame:
        """
        Verarbeitet Lücken-Daten, um zusammenhängende Zeiträume zu identifizieren
        """
        if len(gaps_df) == 0:
            return pl.DataFrame()
            
        # Gruppiere nach IMO und Signal, sortiere nach Zeitstempel
        result = []
        
        # Konvertiere zu Pandas für einfachere Gruppierung und Verarbeitung
        # (In einer produktiven Umgebung kann dies für große Datensätze 
        # effizienter mit Polars-nativen Funktionen implementiert werden)
        gaps_pd = gaps_df.to_pandas()
        
        for (imo, signal), group in gaps_pd.groupby(['imo', 'signal']):
            group = group.sort_values('gap_start')
            
            # Parse timestamps to datetime
            group['gap_start'] = pd.to_datetime(group['gap_start'])
            
            current_start = group['gap_start'].iloc[0]
            prev_time = current_start
            
            for idx, row in group.iloc[1:].iterrows():
                curr_time = row['gap_start']
                
                # Wenn mehr als 15 Minuten zwischen den Zeitstempeln liegen, 
                # betrachte es als neue Lücke
                if (curr_time - prev_time) > timedelta(minutes=15):
                    result.append({
                        'imo': imo,
                        'signal': signal,
                        'gap_start': current_start.isoformat(),
                        'gap_end': prev_time.isoformat(),
                        'loaddate': row['loaddate']
                    })
                    current_start = curr_time
                
                prev_time = curr_time
            
            # Füge die letzte Lücke hinzu
            result.append({
                'imo': imo,
                'signal': signal,
                'gap_start': current_start.isoformat(),
                'gap_end': prev_time.isoformat(),
                'loaddate': group['loaddate'].iloc[-1]
            })
        
        return pl.DataFrame(result)
    
    @staticmethod
    def transform_ships(ships: pl.DataFrame, run_timestamp: str) -> Tuple[pl.DataFrame, Dict[str, pl.DataFrame]]:
        """Transforms ship data and extracts nested tables"""
        ships = ships.lazy().unnest("data")
        
        # Extract nested tables
        tables = {}
        for column, dtype in ships.collect_schema().items():
            if dtype == pl.List(pl.Struct):
                tables[column] = (
                    ships.select("imo", column)
                    .explode(column)
                    .unnest(column)
                    .with_columns(
                        pl.lit(run_timestamp).alias("loaddate")
                    )
                    .collect()
                )
            elif dtype == pl.List:
                tables[column] = (
                    ships.select("imo", column)
                    .explode(column)
                    .with_columns(
                        pl.lit(run_timestamp).alias("loaddate")
                    )
                    .collect()
                )

        # Keep only non-list columns in main table
        ships = ships.select(
            pl.exclude([col for col, dtype in ships.collect_schema().items() if dtype == pl.List])
        ).with_columns(
            pl.lit(run_timestamp).alias("loaddate")
        ).collect()

        return ships, tables
    
    @staticmethod
    def enrich_timeseries_with_friendly_names(timeseries_df: pl.DataFrame, signals_df: pl.DataFrame) -> pl.DataFrame:
        """
        Fügt friendly_name aus der Signaldatei zu den Timeseries-Daten hinzu
        """
        if len(timeseries_df) == 0 or len(signals_df) == 0:
            return timeseries_df
            
        # Extrahiere Signal-Mapping (signal -> friendly_name)
        signal_mapping = (
            signals_df
            .filter(pl.col("friendly_name").is_not_null())
            .select(["signal", "friendly_name"])
            .unique()
        )
        
        # Join mit Timeseries-Daten
        return timeseries_df.join(
            signal_mapping,
            on="signal",
            how="left"
        )


In [8]:
class Pipeline:
    """Main data pipeline class"""
    
    def __init__(self, config: Config, api_key: str):
        self.config = config
        self.api_client = APIClient(config.base_url, api_key, timeout=config.timeout)
        self.processor = DataProcessor()
        self.storage = DataStorage(config)
        
    def process_ship(self, imo: str, run_timestamp: str) -> None:
        """Processes data for a single ship"""
        try:
            # Get and process signals
            _, signals = self.api_client.get_data(f"fleet/{imo}/signals")
            if signals:
                signals_df = pl.DataFrame(signals)
                self.storage.write_file(
                    signals,
                    f"Signals_{imo}",
                    f"{self.config.raw_path}/{run_timestamp}",
                    'json'
                )
                signals_transformed = self.processor.transform_signals(signals_df, run_timestamp)
                self.storage.write_file(
                    signals_transformed,
                    f"Signals_{imo}",
                    f"{self.config.transformed_path}/{run_timestamp}",
                    'parquet'
                )

            # Get and process timeseries
            _, timeseries = self.api_client.get_data(f"fleet/{imo}/timeseries")
            if timeseries:
                ts_df = pl.DataFrame(timeseries)
                self.storage.write_file(
                    timeseries,
                    f"Timeseries_{imo}",
                    f"{self.config.raw_path}/{run_timestamp}",
                    'json'
                )
                # Transformieren und Lücken extrahieren
                ts_transformed, gaps = self.processor.transform_timeseries(ts_df, imo, run_timestamp)
                
                self.storage.write_file(
                    ts_transformed,
                    f"Timeseries_{imo}",
                    f"{self.config.transformed_path}/{run_timestamp}",
                    'parquet'
                )
                
                # Prozessiere und speichere Lücken-Daten, falls vorhanden
                if len(gaps) > 0:
                    processed_gaps = self.processor.process_gaps(gaps)
                    if len(processed_gaps) > 0:
                        self.storage.write_file(
                            processed_gaps,
                            f"Gaps_{imo}",
                            f"{self.config.gaps_path}/{run_timestamp}",
                            'parquet'
                        )
                
        except Exception as e:
            logger.error(f"Failed to process ship {imo}: {str(e)}")
            raise
            
    def run(self, mode: str = "all") -> str:
        """
        Runs the complete data pipeline
        
        Args:
            mode (str): 'all' for complete pipeline, 'timeseries' for only timeseries data,
                        'fleet' for only fleet and signals data
                        
        Returns:
            str: The run timestamp in the format 'YYYY/MM/DD/HH/MM'
        """
        run_start = datetime.now(timezone.utc)
        run_timestamp = run_start.strftime('%Y/%m/%d/%H/%M')
        
        try:
            # Initialize directories
            os.makedirs(f"{self.config.raw_path}/{run_timestamp}", exist_ok=True)
            os.makedirs(f"{self.config.transformed_path}/{run_timestamp}", exist_ok=True)
            os.makedirs(f"{self.config.gaps_path}/{run_timestamp}", exist_ok=True)
            
            # Get ship data
            if mode in ["all", "fleet"]:
                _, ships = self.api_client.get_data("fleet")
                if not ships:
                    raise ValueError("Failed to get ship data")
                    
                # Process ships
                imo_numbers = self.processor.get_imo_numbers(ships)
                
                # Store raw ship data
                self.storage.write_file(
                    ships,
                    'ShipData',
                    f"{self.config.raw_path}/{run_timestamp}",
                    'json'
                )
                
                # Transform and store ship data
                ships_df = pl.DataFrame(ships)
                ships_transformed, tables = self.processor.transform_ships(ships_df, run_timestamp)
                self.storage.write_file(
                    ships_transformed,
                    'ShipData',
                    f"{self.config.transformed_path}/{run_timestamp}",
                    'parquet'
                )
                
                # Process nested tables
                for name, table in tables.items():
                    self.storage.write_file(
                        table,
                        f"ShipData_{name}",
                        f"{self.config.transformed_path}/{run_timestamp}",
                        'parquet'
                    )
                
                # Process signals for all ships
                for imo in imo_numbers:
                    _, signals = self.api_client.get_data(f"fleet/{imo}/signals")
                    if signals:
                        signals_df = pl.DataFrame(signals)
                        self.storage.write_file(
                            signals,
                            f"Signals_{imo}",
                            f"{self.config.raw_path}/{run_timestamp}",
                            'json'
                        )
                        signals_transformed = self.processor.transform_signals(signals_df, run_timestamp)
                        self.storage.write_file(
                            signals_transformed,
                            f"Signals_{imo}",
                            f"{self.config.transformed_path}/{run_timestamp}",
                            'parquet'
                        )
            
            # Process timeseries data
            if mode in ["all", "timeseries"]:
                # Get ship data if not already loaded
                if mode == "timeseries":
                    _, ships = self.api_client.get_data("fleet")
                    if not ships:
                        raise ValueError("Failed to get ship data")
                    imo_numbers = self.processor.get_imo_numbers(ships)
                
                # Process individual ships in parallel
                with ThreadPoolExecutor(max_workers=self.config.max_workers) as executor:
                    # Lambda function to process only timeseries
                    def process_ship_timeseries(imo):
                        try:
                            # Get and process timeseries
                            _, timeseries = self.api_client.get_data(f"fleet/{imo}/timeseries")
                            if timeseries:
                                ts_df = pl.DataFrame(timeseries)
                                self.storage.write_file(
                                    timeseries,
                                    f"Timeseries_{imo}",
                                    f"{self.config.raw_path}/{run_timestamp}",
                                    'json'
                                )
                                # Transformieren und Lücken extrahieren
                                ts_transformed, gaps = self.processor.transform_timeseries(ts_df, imo, run_timestamp)
                                
                                # Combine with historical data (last 5 days)
                                historical_data = self.load_historical_timeseries(imo, self.config.history_days)
                                if len(historical_data) > 0:
                                    # Combine historical and new data
                                    combined = pl.concat([historical_data, ts_transformed])
                                    # Deduplicate
                                    ts_transformed = (
                                        combined
                                        .sort(by=["loaddate"], descending=True)
                                        .unique(subset=["imo", "signal", "signal_timestamp"], keep="first")
                                    )
                                
                                self.storage.write_file(
                                    ts_transformed,
                                    f"Timeseries_{imo}",
                                    f"{self.config.transformed_path}/{run_timestamp}",
                                    'parquet'
                                )
                                
                                # Prozessiere und speichere Lücken-Daten, falls vorhanden
                                if len(gaps) > 0:
                                    processed_gaps = self.processor.process_gaps(gaps)
                                    if len(processed_gaps) > 0:
                                        self.storage.write_file(
                                            processed_gaps,
                                            f"Gaps_{imo}",
                                            f"{self.config.gaps_path}/{run_timestamp}",
                                            'parquet'
                                        )
                        except Exception as e:
                            logger.error(f"Failed to process timeseries for ship {imo}: {str(e)}")
                    
                    # Execute in parallel
                    executor.map(process_ship_timeseries, imo_numbers)
            
            # Clean up old data
            self.storage.cleanup_old_data(self.config.raw_path, self.config.days_to_keep)
            self.storage.cleanup_old_data(self.config.transformed_path, self.config.days_to_keep)
            self.storage.cleanup_old_data(self.config.gaps_path, self.config.days_to_keep)
            
            logger.info(f"Pipeline run completed successfully in {datetime.now(timezone.utc) - run_start}")
            return run_timestamp
                
        except Exception as e:
            logger.error(f"Pipeline failed: {str(e)}")
            raise
        finally:
            runtime = datetime.now(timezone.utc) - run_start
            logger.info(f"Pipeline completed in {runtime}")
            
    def run_timeseries_only(self) -> str:
        """Convenience method to run only the timeseries part of the pipeline"""
        return self.run(mode="timeseries")
        
    def run_fleet_only(self) -> str:
        """Convenience method to run only the fleet part of the pipeline"""
        return self.run(mode="fleet")
    
    def load_historical_timeseries(self, imo: str, days: int = 5) -> pl.DataFrame:
        """
        Lädt historische Timeseries-Daten für ein Schiff aus den letzten n Tagen
        """
        try:
            today = datetime.now()
            
            # Erstelle eine Liste von Pfaden für die letzten n Tage
            paths = []
            for i in range(days):
                check_date = today - timedelta(days=i)
                date_path = check_date.strftime('%Y/%m/%d')
                
                # Alle Unterordner des Tages durchsuchen (Stunden/Minuten)
                full_path = Path(f"{self.config.transformed_path}/{date_path}")
                
                if full_path.exists():
                    # Finde die letzten Runs des Tages
                    for hour_dir in sorted(full_path.glob("*"), reverse=True):
                        if hour_dir.is_dir():
                            for minute_dir in sorted(hour_dir.glob("*"), reverse=True):
                                if minute_dir.is_dir():
                                    file_path = minute_dir / f"Timeseries_{imo}.parquet"
                                    if file_path.exists():
                                        paths.append(str(file_path))
                                        # Nimm nur den letzten Run des Tages
                                        break
                            # Nimm nur die letzte Stunde des Tages
                            break
            
            # Load and combine data
            dfs = []
            for path in paths:
                try:
                    df = pl.read_parquet(path)
                    if len(df) > 0:
                        dfs.append(df)
                except Exception as e:
                    logger.warning(f"Failed to read {path}: {str(e)}")
            
            if not dfs:
                logger.info(f"No historical timeseries data found for ship {imo}")
                return pl.DataFrame()
        except Exception as e:
            logger.error(f"Failed to load historical timeseries for ship {imo}: {str(e)}")
            return pl.DataFrame()
        
    def get_all_signals(self, run_timestamp: str) -> pl.DataFrame:
        """
        Sammelt alle Signaldefinitionen aus den aktuellen Daten
        """
        try:
            signals_path = Path(f"{self.config.transformed_path}/{run_timestamp}")
            all_signals = []
            
            for file in signals_path.glob("Signals_*.parquet"):
                try:
                    df = pl.read_parquet(file)
                    if len(df) > 0:
                        # Extrahiere nur die relevanten Spalten für das Mapping
                        signals_mapping = df.select([
                            "signal", "friendly_name", "unit", "object_code", "name_code", "group_name", "sub_group"
                        ])
                        all_signals.append(signals_mapping)
                except Exception as e:
                    logger.warning(f"Failed to read {file}: {str(e)}")
            
            if not all_signals:
                logger.warning("No signal definitions found")
                return pl.DataFrame()
                
            # Combine all signal definitions and deduplicate
            return pl.concat(all_signals).unique(subset=["signal"], keep="first")
            
        except Exception as e:
            logger.error(f"Failed to collect signal definitions: {str(e)}")
            return pl.DataFrame()
        
    def process_and_store_to_db(self, engine: sa.Engine, run_timestamp: str) -> None:
        """
        Processes data and stores it to the SQL database
        
        This method:
        1. Loads timeseries data from the current run
        2. Enriches it with friendly names
        3. Writes the combined data to the SQL database using the specialized method
        4. Writes gaps data to a separate table
        """
        try:
            # 1. Load timeseries data from current run
            transformed_path = Path(f"{self.config.transformed_path}/{run_timestamp}")
            
            # Find all timeseries parquet files
            timeseries_files = list(transformed_path.glob("Timeseries_*.parquet"))
            if not timeseries_files:
                logger.warning(f"No timeseries files found in {run_timestamp}")
                return
            
            # 2. Load and combine all timeseries data
            ts_dfs = []
            for file in timeseries_files:
                try:
                    df = pl.read_parquet(file)
                    if len(df) > 0:
                        ts_dfs.append(df)
                except Exception as e:
                    logger.warning(f"Failed to read {file}: {str(e)}")
            
            if not ts_dfs:
                logger.warning("No data loaded from timeseries files")
                return
            
            combined_ts = pl.concat(ts_dfs)
            logger.info(f"Combined {len(ts_dfs)} timeseries files with {len(combined_ts)} rows total")
            
            # 3. Get all signal definitions for enrichment
            all_signals = self.get_all_signals(run_timestamp)
            
            # 4. Enrich timeseries data with friendly names
            if len(all_signals) > 0:
                combined_ts = self.processor.enrich_timeseries_with_friendly_names(combined_ts, all_signals)
                logger.info("Enriched timeseries data with friendly names")
            
            # 5. Write to database using the specialized method
            if len(combined_ts) > 0:
                self.storage.write_ts_to_msdb(combined_ts, engine)
                logger.info("Successfully processed and stored timeseries data to database")
            
            # 6. Process and store gaps data
            gaps_path = Path(f"{self.config.gaps_path}/{run_timestamp}")
            if gaps_path.exists():
                gaps_files = list(gaps_path.glob("Gaps_*.parquet"))
                gaps_dfs = []
                
                for file in gaps_files:
                    try:
                        df = pl.read_parquet(file)
                        if len(df) > 0:
                            gaps_dfs.append(df)
                    except Exception as e:
                        logger.warning(f"Failed to read gaps file {file}: {str(e)}")
                
                if gaps_dfs:
                    combined_gaps = pl.concat(gaps_dfs)
                    logger.info(f"Combined {len(gaps_dfs)} gaps files with {len(combined_gaps)} rows total")
                    
                    # Write gaps data to a separate table
                    self.storage.write_to_db(combined_gaps, engine, "TimeSeries_Gaps", if_exists="append")
                    logger.info("Successfully stored gaps data to database")
            
        except Exception as e:
            logger.error(f"Failed to process and store data to database: {str(e)}")
            raise
                
            # Combine all dataframes and keep only the latest value for each combination
            combined = pl.concat(dfs)
            
            # Dedupliziere die Daten - behalte nur den neuesten Eintrag für jede Kombination von imo, signal und signal_timestamp
            deduplicated = (
                combined
                .sort(by=["loaddate"], descending=True)
                .unique(subset=["imo", "signal", "signal_timestamp"], keep="first")
            )
            
            return deduplicated
            
        except Exception as e:
            logger.error(f"Failed to load historical timeseries for ship {imo}: {str(e)}")
            return pl.DataFrame()

## Pipeline Starten
Hauptfunktionsaufruf für die Pipeline

mode (str): 'all', 'timeseries', oder 'fleet'

In [9]:
mode = "all"

logger = setup_logging()

# Load environment variables
load_dotenv()

True

In [10]:
# Check if environment variables are set
api_key = os.getenv('HOPPE_API_KEY')
if not api_key:
    logger.error("HOPPE_API_KEY environment variable not set")

In [11]:
# Configure pipeline
config = Config(
    base_url=os.getenv('HOPPE_BASE_URL', "https://api.hoppe-sts.com/"),
    raw_path=os.getenv('RAW_PATH', "./data/raw_data"),
    transformed_path=os.getenv('TRANSFORMED_PATH', "./data/transformed_data"),
    gaps_path=os.getenv('GAPS_PATH', "./data/gaps_data"),
    max_workers=int(os.getenv('MAX_WORKERS', "8")),
    retry_attempts=int(os.getenv('RETRY_ATTEMPTS', "5")),
    timeout=int(os.getenv('TIMEOUT', "45")),
    days_to_keep=int(os.getenv('DAYS_TO_KEEP', "90")),
    history_days=int(os.getenv('HISTORY_DAYS', "5"))
)

In [12]:

# Create and run pipeline
pipeline = Pipeline(config, api_key)
run_timestamp = pipeline.run(mode)

2025-03-10 23:46:03,661 - hoppe_etl_pipeline - INFO - Data saved to ./data/raw_data/2025/03/10/22/45/ShipData.json
2025-03-10 23:46:03,862 - hoppe_etl_pipeline - INFO - Data saved to ./data/transformed_data/2025/03/10/22/45/ShipData.parquet
2025-03-10 23:46:03,882 - hoppe_etl_pipeline - INFO - Data saved to ./data/transformed_data/2025/03/10/22/45/ShipData_main_engines.parquet
2025-03-10 23:46:03,910 - hoppe_etl_pipeline - INFO - Data saved to ./data/transformed_data/2025/03/10/22/45/ShipData_aux_engines.parquet
2025-03-10 23:46:10,415 - hoppe_etl_pipeline - INFO - Data saved to ./data/raw_data/2025/03/10/22/45/Signals_9400071.json
2025-03-10 23:46:10,465 - hoppe_etl_pipeline - INFO - Data saved to ./data/transformed_data/2025/03/10/22/45/Signals_9400071.parquet
2025-03-10 23:46:10,676 - hoppe_etl_pipeline - INFO - Data saved to ./data/raw_data/2025/03/10/22/45/Signals_9725512.json
2025-03-10 23:46:10,698 - hoppe_etl_pipeline - INFO - Data saved to ./data/transformed_data/2025/03/10/22

In [ ]:
# If database connection is configured, store to database
db_connection_string = os.getenv('MSSQL_CONNECTION_STRING')
if db_connection_string:
    try:
        engine = sa.create_engine(db_connection_string)
        logger.info("Database connection established")
        
        # Process and store data to database
        pipeline.process_and_store_to_db(engine, run_timestamp)
        logger.info("Database integration completed successfully")
    except Exception as e:
        logger.error(f"Database integration failed: {str(e)}")
else:
    logger.warning("MSSQL_CONNECTION_STRING not set, skipping database integration")


In [13]:
def find_timeseries_files(base_path: str, max_days: int = None) -> list:
    """
    Findet alle Timeseries-Parquet-Dateien im angegebenen Verzeichnis
    
    Args:
        base_path: Basispfad zum Durchsuchen
        max_days: Optional, maximale Anzahl der zu berücksichtigenden Tage (neueste zuerst)
        
    Returns:
        Liste von Pfaden zu Timeseries-Dateien
    """
    logger.info(f"Durchsuche Verzeichnis {base_path} nach Timeseries-Dateien")
    
    base_dir = Path(base_path)
    if not base_dir.exists() or not base_dir.is_dir():
        logger.error(f"Verzeichnis {base_path} existiert nicht oder ist kein Verzeichnis")
        return []
    
    # Finde alle Jahr-Verzeichnisse, sortiere absteigend für neueste zuerst
    years = sorted([d for d in base_dir.glob("*") if d.is_dir() and d.name.isdigit()], 
                   key=lambda x: x.name, reverse=True)
    
    all_files = []
    days_processed = 0
    
    # Durchlaufe Jahre, Monate, Tage
    for year_dir in years:
        months = sorted([d for d in year_dir.glob("*") if d.is_dir() and d.name.isdigit()], 
                        key=lambda x: x.name, reverse=True)
        
        for month_dir in months:
            days = sorted([d for d in month_dir.glob("*") if d.is_dir() and d.name.isdigit()], 
                          key=lambda x: x.name, reverse=True)
            
            for day_dir in days:
                if max_days is not None and days_processed >= max_days:
                    break
                
                # Finde alle Stunden und Minuten für diesen Tag
                ts_files = []
                hour_dirs = sorted([d for d in day_dir.glob("*") if d.is_dir()], 
                                   key=lambda x: x.name, reverse=True)
                
                for hour_dir in hour_dirs:
                    minute_dirs = sorted([d for d in hour_dir.glob("*") if d.is_dir()], 
                                         key=lambda x: x.name, reverse=True)
                    
                    for minute_dir in minute_dirs:
                        # Finde alle Timeseries-Dateien in diesem Minuten-Verzeichnis
                        files = list(minute_dir.glob("Timeseries_*.parquet"))
                        if files:
                            ts_files.extend(files)
                            # Nur der letzte Run des Tages
                            break
                    
                    if ts_files:
                        # Nur die letzte Stunde des Tages
                        break
                
                all_files.extend(ts_files)
                days_processed += 1
                
                if max_days is not None and days_processed >= max_days:
                    logger.info(f"Maximale Anzahl von Tagen ({max_days}) erreicht")
                    break
            
            if max_days is not None and days_processed >= max_days:
                break
        
        if max_days is not None and days_processed >= max_days:
            break
    
    logger.info(f"Gefunden: {len(all_files)} Timeseries-Dateien aus {days_processed} Tagen")
    return all_files

In [14]:
def load_and_combine_timeseries(file_paths: list) -> pl.DataFrame:
    """
    Lädt und kombiniert alle Timeseries-Dateien
    
    Args:
        file_paths: Liste der zu ladenden Dateipfade
        
    Returns:
        Polars DataFrame mit kombinierten Daten
    """
    logger.info(f"Lade {len(file_paths)} Timeseries-Dateien")
    
    all_dfs = []
    total_rows = 0
    
    for idx, file_path in enumerate(file_paths):
        try:
            if idx % 10 == 0:
                logger.info(f"Verarbeite Datei {idx + 1} von {len(file_paths)}")
            
            df = pl.read_parquet(file_path)
            rows = len(df)
            total_rows += rows
            
            if rows > 0:
                all_dfs.append(df)
        except Exception as e:
            logger.error(f"Fehler beim Lesen von {file_path}: {str(e)}")
    
    if not all_dfs:
        logger.warning("Keine Daten geladen")
        return pl.DataFrame()
    
    # Kombiniere alle DataFrames
    combined_df = pl.concat(all_dfs)
    logger.info(f"Insgesamt {total_rows} Zeilen geladen, {len(combined_df)} vor Deduplizierung")
    
    return combined_df

In [15]:
def clean_and_deduplicate(df: pl.DataFrame) -> pl.DataFrame:
    """
    Entfernt Nullwerte und Duplikate aus dem DataFrame
    
    Args:
        df: Zu bereinigender DataFrame
        
    Returns:
        Bereinigter DataFrame
    """
    if len(df) == 0:
        return df
    
    # Entferne Zeilen mit Nullwerten
    no_nulls = df.filter(pl.col("signal_value").is_not_null())
    logger.info(f"Zeilen nach Entfernen von Nullwerten: {len(no_nulls)}")
    
    # Sortiere nach Ladedatum (absteigend) und entferne Duplikate
    deduplicated = (
        no_nulls
        .sort(by=["loaddate"], descending=True)
        .unique(subset=["imo", "signal", "signal_timestamp"], keep="first")
    )
    
    logger.info(f"Zeilen nach Deduplizierung: {len(deduplicated)}")
    return deduplicated

def pivot_timeseries(df: pl.DataFrame, max_signals: int = None) -> pl.DataFrame:
    """
    Pivotisiert die Timeseries-Daten: Signal wird zu Spalten
    
    Args:
        df: Zu pivotisierender DataFrame
        max_signals: Optional, maximale Anzahl von Signalen zu verarbeiten
                    (für Speicher- und Leistungsoptimierung)
        
    Returns:
        Pivotisierter DataFrame
    """
    if len(df) == 0:
        return df
    
    logger.info("Pivotisiere Daten")
    
    # Identifiziere eindeutige Signale
    unique_signals = df.select("signal").unique()
    signal_count = len(unique_signals)
    
    logger.info(f"Gefunden: {signal_count} eindeutige Signale")
    
    if max_signals is not None and signal_count > max_signals:
        logger.warning(f"Zu viele Signale ({signal_count}), begrenze auf {max_signals}")
        
        # Verwende die häufigsten Signale, wenn zu viele vorhanden sind
        signal_counts = df.group_by("signal").count().sort(by="count", descending=True)
        top_signals = signal_counts.head(max_signals).select("signal")
        
        # Filtere DataFrame auf die häufigsten Signale
        df = df.join(top_signals, on="signal")
        logger.info(f"DataFrame auf {len(df)} Zeilen mit Top-{max_signals} Signalen reduziert")
    
    try:
        # Pivotisiere den DataFrame
        pivoted = df.pivot(
            values="signal_value",
            index=["imo", "signal_timestamp", "loaddate"],
            columns="signal"
        )
        
        logger.info(f"Pivotisierter DataFrame hat {len(pivoted)} Zeilen und {len(pivoted.columns)} Spalten")
        return pivoted
    except Exception as e:
        logger.error(f"Fehler bei der Pivotisierung: {str(e)}")
        
        # Alternative Methode mit expliziter Speicherverwaltung
        logger.info("Versuche alternative Pivotisierungsmethode...")
        
        # Gruppieren nach imo, timestamp, loaddate
        pivoted_dfs = []
        
        for sig in df.select("signal").unique().to_series():
            try:
                # Filtern für dieses Signal
                signal_df = df.filter(pl.col("signal") == sig)
                
                # Umbenennen der signal_value-Spalte zum Signalnamen
                renamed = signal_df.select(
                    ["imo", "signal_timestamp", "loaddate", 
                     pl.col("signal_value").alias(sig)]
                )
                
                pivoted_dfs.append(renamed)
            except Exception as sub_e:
                logger.error(f"Fehler bei der Verarbeitung von Signal {sig}: {str(sub_e)}")
        
        if not pivoted_dfs:
            logger.error("Keine Daten nach alternativer Pivotisierung")
            return pl.DataFrame()
        
        # Join all signal dataframes
        result = pivoted_dfs[0]
        for idx, signal_df in enumerate(pivoted_dfs[1:], 1):
            if idx % 10 == 0:
                logger.info(f"Verbinde Signal {idx} von {len(pivoted_dfs) - 1}")
                
            try:
                result = result.join(
                    signal_df, 
                    on=["imo", "signal_timestamp", "loaddate"], 
                    how="outer"
                )
            except Exception as join_e:
                logger.error(f"Fehler beim Verbinden von Signal {idx}: {str(join_e)}")
        
        logger.info(f"Alternative Pivotisierung: {len(result)} Zeilen, {len(result.columns)} Spalten")
        return result

In [20]:
start_time = datetime.now()
logger.info(f"Starte Verarbeitung um {start_time}")

base_path = "./data/transformed_data"
output = "./pivoted_timeseries.parquet"
max_days = None
max_signals = None

2025-03-10 23:53:13,252 - hoppe_etl_pipeline - INFO - Starte Verarbeitung um 2025-03-10 23:53:13.252570


In [21]:
# Finde Dateien
ts_files = find_timeseries_files(base_path, max_days)

if not ts_files:
    logger.error("Keine Timeseries-Dateien gefunden")

# Lade und kombiniere Daten
combined_df = load_and_combine_timeseries(ts_files)

if len(combined_df) == 0:
    logger.error("Keine Daten geladen")

# Bereinige Daten
clean_df = clean_and_deduplicate(combined_df)

if len(clean_df) == 0:
    logger.error("Keine Daten nach Bereinigung")

# Pivotisiere Daten
pivoted_df = pivot_timeseries(clean_df, max_signals)

if len(pivoted_df) == 0:
    logger.error("Keine Daten nach Pivotisierung")


2025-03-10 23:53:19,437 - hoppe_etl_pipeline - INFO - Durchsuche Verzeichnis ./data/transformed_data nach Timeseries-Dateien
2025-03-10 23:53:19,454 - hoppe_etl_pipeline - INFO - Gefunden: 10 Timeseries-Dateien aus 1 Tagen
2025-03-10 23:53:19,455 - hoppe_etl_pipeline - INFO - Lade 10 Timeseries-Dateien
2025-03-10 23:53:19,456 - hoppe_etl_pipeline - INFO - Verarbeite Datei 1 von 10
2025-03-10 23:53:19,957 - hoppe_etl_pipeline - INFO - Insgesamt 1235318 Zeilen geladen, 1235318 vor Deduplizierung
2025-03-10 23:53:19,959 - hoppe_etl_pipeline - INFO - Zeilen nach Entfernen von Nullwerten: 1235318
2025-03-10 23:53:20,368 - hoppe_etl_pipeline - INFO - Zeilen nach Deduplizierung: 1235318
2025-03-10 23:53:20,370 - hoppe_etl_pipeline - INFO - Pivotisiere Daten
2025-03-10 23:53:20,434 - hoppe_etl_pipeline - INFO - Gefunden: 218 eindeutige Signale
C:\Users\Privat\AppData\Local\Temp\ipykernel_29988\2044662848.py:64: DeprecationWarning: The argument `columns` for `DataFrame.pivot` is deprecated. It 

In [22]:
# Speichere Ergebnis
try:
    # Erstelle das Ausgabeverzeichnis, falls es nicht existiert
    output_dir = os.path.dirname(output)
    if output_dir:
        os.makedirs(output_dir, exist_ok=True)
        
    pivoted_df.write_parquet(output, compression="snappy")
    logger.info(f"Pivotisierte Daten gespeichert nach {output}")
    
    # Berechne Statistiken
    stats = {
        "Anzahl der Zeilen": len(pivoted_df),
        "Anzahl der Spalten": len(pivoted_df.columns),
        "Eindeutige IMOs": len(pivoted_df.select("imo").unique()),
        "Zeitraum": f"{pivoted_df.select('signal_timestamp').min()[0]} bis {pivoted_df.select('signal_timestamp').max()[0]}"
    }
    
    logger.info("Statistiken der pivotisierten Daten:")
    for key, value in stats.items():
        logger.info(f"  {key}: {value}")
    
except Exception as e:
    logger.error(f"Fehler beim Speichern der Daten: {str(e)}")

# Beende Zeitmessung
end_time = datetime.now()
duration = end_time - start_time
logger.info(f"Verarbeitung abgeschlossen nach {duration}, Endzeit: {end_time}")

2025-03-10 23:54:20,425 - hoppe_etl_pipeline - INFO - Pivotisierte Daten gespeichert nach ./pivoted_timeseries.parquet
2025-03-10 23:54:20,471 - hoppe_etl_pipeline - INFO - Statistiken der pivotisierten Daten:
2025-03-10 23:54:20,472 - hoppe_etl_pipeline - INFO -   Anzahl der Zeilen: 11556
2025-03-10 23:54:20,475 - hoppe_etl_pipeline - INFO -   Anzahl der Spalten: 221
2025-03-10 23:54:20,478 - hoppe_etl_pipeline - INFO -   Eindeutige IMOs: 10
--- Logging error ---
Traceback (most recent call last):
  File "D:\Python311\Lib\logging\__init__.py", line 1113, in emit
    stream.write(msg + self.terminator)
  File "D:\Python311\Lib\encodings\cp1252.py", line 19, in encode
    return codecs.charmap_encode(input,self.errors,encoding_table)[0]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
UnicodeEncodeError: 'charmap' codec can't encode characters in position 81-108: character maps to <undefined>
Call stack:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File 